# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [17]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [18]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [19]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [20]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [21]:
def foldl2(f, x0, lst):
    return foldr(lambda a, temp: lambda x: temp(f(x, a)), lambda x : x, lst)(x0)

In [22]:
def foldr2(f, x0, lst):
    return foldl(lambda temp, a: lambda x: temp(f(a, x)), lambda x : x, lst)(x0)

In [23]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3]), \
foldr2(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [24]:
def check_inv(a, b):
    n = len(a)
    l = len(b)
    if (n == 0 or l == 0 or n<l):
        return False
    
    def addLetter(d, c):
        if c not in d.keys():
            d[c] = 1
        else:
            d[c] += 1
            
    def delLetter(d, c):        
        if d[c] == 1:
            d.pop(c)
        else:
            d[c] -= 1
    
    dictA = {}
    dictB = {}
    
    for i in range(l):
        addLetter(dictA, a[i])
        addLetter(dictB, b[i])
    
    for i in range(l, n):
        if (dictA == dictB):
            return True
        delLetter(dictA, a[i-l])
        addLetter(dictA, a[i])
    return False


check_inv('abcrotm', 'tro')
check_inv('abbcq', 'abcc')

False

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [2]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left == None and self.right== None:
            yield self.value
        if self.left:
            yield from iter(self.left)
        if self.right:
            yield from iter(self.right)
    
    def __str__(self):
        return '({},{},{})'.format(self.value, str(self.left), str(self.right))
    
    
    def __repr__(self):
        return 'Tree({},{},{})'.format(self.value, repr(self.left), repr(self.right))

    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [1]:
import re
from operator import add, mul, sub, truediv


operations = {'*': mul, '\\': truediv, '+': add, '-': sub}

def calc1(expr):
    while True:
        firstPrior = re.findall(r'\d+ [\*|\/] \d+', expr)
        if firstPrior == []:
            break
        for i in firstPrior:
            k = i.split(' ')
            expr = expr.replace(i, str(operations[k[1]](int(k[0]), int(k[2]))))
    
    while True:
        secondPrior = re.findall(r'\d+ [\+|\-] \d+', expr)
        if secondPrior == []:
            break
        for i in secondPrior:
            k = i.split(' ')
            expr = expr.replace(i, str(operations[k[1]](int(k[0]), int(k[2]))))
    
    return expr


def calc(expr):
    while True:
        brackets = re.findall(r"\(([^\(\)]+)\)", expr)
        if brackets == []:
            break
        for i in brackets:
            expr = expr.replace('('+i+')', calc1(i))
    
    return int(calc1(expr))

calc('2 * (15 - 3 * 4) - 2') == 4

True